# Putting it All Together: Optimizing Allocation and Transportation
Before we build our final model, let's import the necessary packages again and refresh the data. 

In [1]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

distanceMatrix_scenarios = pd.read_csv("data/distanceMatrix_scenarios.csv")
warehouses = pd.read_csv("data/warehouses.csv")
disasters = pd.read_csv("data/disasters.csv")
# one bucket is needed for 2.5 people on average
total_buckets = warehouses["Buckets"].sum()
disasters["demand"] = (
    (disasters["People Impacted"] / 2.5).round().clip(upper=total_buckets).astype(int)
)
demand = disasters["demand"].values
probs = [1 / len(demand)] * len(demand)

Great, now that we can solve for the time it takes to address every single disaster, we can finally answer the original question posed: How do we best allocate supplies to all the warehouses? The key here is to extend the optimization model. Remember how in our constraints we included that you can't send more than the warehouse has? 
$$y^k_i \leq b_i,~  \forall i \in I \quad \text{(you can't send more than a warehouse has)}$$

So far, we've just been using the actual allocation we have at each warehouse. But what if those changes? Suddenly we might have an entirely new solution. So, we say that the model's solution (i.e., the allocations $y^k_i \in Y$) is some function $Y=f(X)$ based on our warehouse starting inventories $x_i \in X$.

While this may initially look intimidating, it is one of the easiest changes to make to our current model. All we are doing is creating a decision variable $x_i \ge 0$ for each warehouse $i \in I$ instead of using constant values and limit the total amount of supplies over all warehouses by $\chi$.
Let's substitute back in our model from the last section with the new constraints to see this firsthand.

$$
\min_{x,y} \sum_k P^k \sum_i \tau_{ij}\cdot y^k_i
$$

Then all we need to do is update the constraints. I've included the line to make it easier to see what's new as our list grows. It has no mathematical significance. 
$$
\begin{aligned}
\text{s.t.}  \sum_{i} y^k_{i} & = d^k & \forall k \in K & \quad\text{(total supplies sent must meet demand)}\\
y^k_i & \leq x_i & \forall i \in I, \forall k \in K & \quad \text{(you can't send more than a warehouse has)}\\
y^k_{i} & \geq 0 & \forall i \in I, \forall k \in K & \quad \text{(you can't send negative supplies)}\\
\hline \\
\sum_{i} x_i & = \chi & & \quad \text{(we allocate all supplies and no more)}\\
x_{i} & \geq 0 & \forall i \in I & \quad \text{(you can't allocate negative supplies)}
\end{aligned}
$$
So how do we implement this in Gurobi?

In [2]:
m = len(demand)  # number of disasters/scenarios
t = distanceMatrix_scenarios.set_index(["scenario", "warehouse"]).squeeze().to_dict()
b = warehouses["Buckets"][warehouses["Buckets"] > 0].reset_index(drop=True)
n = len(b)  # number of warehouses

model_full = gp.Model("full_allocation")

# Amount to take per warehouse
y = model_full.addVars(m, n, name="Warehouse_Allocation")

# Warehouse inventories
x = model_full.addVars(n, name="Warehouse_Inventory")

# Total national endowment constraint
model_full.addConstr(x.sum() == b.sum(), name="Total_National_Endowment")

# Add constraints to meet demand for each disaster scenario (k)
for k in range(m):

    model_full.addConstr(y.sum(k, "*") == demand[k], name=f"Meet_Demand_K:{k}")

    # Cannot allocate more than amount assigned to warehouse
    # x is now a decision variable
    for i in range(n):
        model_full.addConstr(y[k, i] <= x[i], name=f"warehouse_endowment_K:{k}_I:{i}")

# Objective function to minimize the weighted driving time using t as a parameter
objective = gp.quicksum(
    probs[k] * gp.quicksum(t[k, i] * y[k, i] for i in range(n)) for k in range(m)
)

# Optimize model
model_full.setObjective(objective, GRB.MINIMIZE)
model_full.optimize()

Restricted license - for non-production use only - expires 2025-11-24
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[rosetta2] - Darwin 24.4.0 24E263)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 485 rows, 483 columns and 1407 nonzeros
Model fingerprint: 0x8817cae8
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e-02, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 4e+04]
Presolve time: 0.05s
Presolved: 485 rows, 483 columns, 1407 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.4281409e+04   5.415380e+05   0.000000e+00      0s
     272    3.4027302e+05   0.000000e+00   0.000000e+00      0s

Solved in 272 iterations and 0.07 seconds (0.00 work units)
Optimal objective  3.402730155e+05


In [3]:
warehouse_inventories = model_full.getAttr("X", x)
warehouse_inventories

{0: 0.0,
 1: 18673.0,
 2: 0.0,
 3: 0.0,
 4: 0.0,
 5: 0.0,
 6: 0.0,
 7: 0.0,
 8: 0.0,
 9: 725.0,
 10: 0.0,
 11: 0.0,
 12: 0.0,
 13: 0.0,
 14: 0.0,
 15: 11289.0,
 16: 0.0,
 17: 0.0,
 18: 0.0,
 19: 0.0,
 20: 10124.0}

In [4]:
y_values_full = pd.DataFrame(
    {
        "Warehouse allocation": model_full.getAttr("VarName", y),
        "Buckets": model_full.getAttr("X", y),
    }
)
y_values_full

Warehouse allocation  Buckets
0  0     Warehouse_Allocation[0,0]      0.0
   1     Warehouse_Allocation[0,1]  18673.0
   2     Warehouse_Allocation[0,2]      0.0
   3     Warehouse_Allocation[0,3]      0.0
   4     Warehouse_Allocation[0,4]      0.0
...                            ...      ...
21 16  Warehouse_Allocation[21,16]      0.0
   17  Warehouse_Allocation[21,17]      0.0
   18  Warehouse_Allocation[21,18]      0.0
   19  Warehouse_Allocation[21,19]      0.0
   20  Warehouse_Allocation[21,20]  10124.0

[462 rows x 2 columns]

## A Few Final Tidbits

Now that we have our overall framework, we can extend it fairly easily to better model real-life scenarios. Let's look at two final pieces of the puzzle that ESUPS considers in their model.

### Cost

Along with how long it takes to get items to a disaster relief site, it's also important to consider the cost to accomplish it. It might be a few hours faster to charter a jet to deliver blankets in the aftermath of a disaster, however, if it is 100x more expensive than by truck, that may constrain the organization from buying more blankets, chartering more trucks, or making it difficult to resupply for future disasters. So just as we solve for ways to minimize time, it can be important for firms with limited resources to make sure their money is being used to do the most good it can.

So how do we do this? It's fairly simple. Our time matrix, which we've been using to show how close or far buildings are from the disaster relief site, is just a set of predefined weights/discounts. So, if we change the numbers to reflect the cost of transit, then suddenly we're solving a cost-minimization problem! In fact, the substitution is so one-to-one, that besides switching $\tau_{ik}$ for $c_{ik}$, we don't have to change the model.

### Travel Mode

The second additional facet considered in our real-life model that we haven't encountered yet is transportation mode. We alluded to it a little in the cost section, but often there is the option to fly or ship goods to a region, which can be especially useful when far away or the roads are clogged or otherwise unusable (such is often the case after a disaster).

So how do we implement this? Well let's take a look back at $y_i^k$, our variable which says how many goods to send from warehouse $i$ to disaster $k$. All we want to do is reflect and updated description: how many goods to send from warehouse $i$ to disaster $k$ via mode $r$. This can easily be represented as $y_{ir}^k$, let's explain what's happened. Before, $y$ was an array $K$ with each index holding a sub array $I$ (which we could also write as $K \times I$), now each index in our subarrays also has an array $R$ with size 3 to represent how much is sent via truck, plane, or boat. So our final 3-dimensional array is $K \times I \times R$. This may seem intimidating at first, but remember, adding a dimension just means adding one more nested for-loop!

Let's look at how we would implement this. Remember, from a math point of view, all we've done is say $y_i^k$ can be broken down into $3$ modes instead of 1. So, it's rewritten as
$$
\min_{x,y} \sum_k P^k \sum_i \sum_r \tau_{ikr}\cdot y^k_{ir}
$$

Then all we need to do is update the constraints:
$$
\begin{aligned}
\text{s.t.} \quad \sum_{i}\sum_{r} y^k_{ir} & = d^k & \forall k \in K & \quad \text{(total supplies sent must meet demand)}\\
\sum_{r} y^k_{ir} & \leq x_i & \forall i \in I,~\forall k \in K & \quad \text{(you can't send more than a warehouse has)}\\
y^k_{ir} & \geq 0 & \forall r \in R,~ \forall i \in I,~ \forall k \in K & \quad \text{(you can't send negative supplies)}\\
\sum_{i} x_i & = \chi & & \quad \text{(we allocate all supplies and no more)}\\
x_{i} & \geq 0 & \forall i \in I & \quad \text{(you can't allocate negative supplies)}\\
\end{aligned}
$$
We've added a few more sums here, but remember, in math, a sum is just a for loop. $\sum_r$ is the equivalent to `for r in R:`. So how would we implement it in the format we've been using so far? This is going to be left as an open-ended exercise to the reader! If you get stuck, you can reference the production solver we'll be exploring in one of the next notebooks, which includes the mode of travel but is set up in a different approach than we've been using so far!

### Open-Ended Implementation
**data needed for this, it's in the works!**

In [5]:
# Write your code here

## Interpreting the Solution 

So let's look at how much slower our real life allocations are in comparison to the optimal solution. How can we do this comparison? Let's consider some metrics!

### Enhancing System Performance with the Balance Metric

In humanitarian logistics, the efficiency of inventory allocation directly impacts the ability to respond swiftly and cost-effectively to disasters. The **Balance Metric** $(D)$ is a critical tool developed to evaluate the alignment of current inventory distribution with an optimal allocation. This metric is particularly valuable in contexts where multiple organizations independently manage inventory across various depots, without a centralized coordination mechanism.

### Definition and Calculation of the Balance Metric

The balance metric $(D)$ is defined as the ratio between the actual objective value (either cost or time) of the current inventory allocation $V(A)$ and the optimal objective value $V(A^*)$ given the same overall capacity:
$$ D = \frac{V(A)}{V(A^*)} $$

Here:
-  $V(A)$: Represents the current system-wide cost or time to meet demand based on the existing inventory allocation \(X\).
-  $V(A^*)$: Represents the minimized cost or time if the inventory were optimally distributed across all depots.

### Interpretation of the Balance Metric

1. **Optimal Inventory Allocation**:
   The optimal value of $D$ is 1. This occurs when the current allocation perfectly aligns with the optimal allocation, meaning no further reallocation could reduce costs or response times.

2. **Identifying Imbalances**:
   When $D > 1$, the system is considered "out-of-balance." A value of 1.2, for example, implies that the current allocation incurs 20% higher costs or longer response times compared to an optimal arrangement. This indicates a potential for improvement by reallocating resources more effectively.

3. **Guiding System Improvements**:
   The balance metric is not only an indicator of inefficiency but also a guide for decision-making. By identifying locations or items with the highest imbalance, decision-makers can prioritize inventory reallocations that would yield the most significant improvements in terms of cost savings or faster response times.

### Practical Applications in Humanitarian Logistics

The balance metric offers several practical applications for optimizing humanitarian response efforts:

- **Strategic Reallocation of Resources**:
  Organizations can use the balance metric to identify under-stocked or over-stocked depots and adjust inventory levels accordingly. This strategic reallocation can significantly enhance response times or reduce costs, especially in multi-organizational contexts where coordination is limited.

- **Sensitivity to Network Changes**:
  The balance metric is responsive to changes in the logistics network. For example, if a new depot is added in a high-risk area and remains under-stocked, the balance metric will reflect this imbalance, prompting an assessment of whether inventory should be redistributed to better leverage the new depot.

- **Decision-Making in Real-Time Operations**:
  By continuously monitoring the balance metric as part of a real-time dashboard, operational managers can be alerted to changes that may impact overall system performance. This enables them to make data-driven decisions quickly, improving the overall resilience and responsiveness of the humanitarian supply chain.

### Limitations and Considerations

While the balance metric provides valuable insights into inventory allocation efficiency, it is important to consider its limitations:

- **Impact of Extreme Events**:
  The balance metric can be influenced by extreme scenarios, such as very large-scale disasters that significantly impact the calculated demand. As a result, it should be interpreted alongside other metrics, such as the fraction of demand served or the weighted fraction of disasters completely served, to provide a more comprehensive picture of system performance.

- **Dependence on Data Quality and Model Assumptions**:
  The accuracy of the balance metric depends on the quality of input data and the assumptions made in the model. Ensuring robust and accurate data collection processes and regularly updating model parameters to reflect real-world conditions are essential for maintaining the reliability of the metric.

### Conclusion

The balance metric $D$ offers a powerful tool for evaluating and improving the efficiency of inventory allocation in humanitarian logistics. By identifying imbalances and guiding strategic reallocation decisions, this metric can help organizations optimize their response efforts, ensuring that resources are used most effectively to meet the needs of affected populations during disasters.

## Data Science Extension

We've seen how the objective function uses the uniform distribution to solve the problem. But what if knowing that climate change is increasingly energizing large storms, we decide the past hurricane impacts aren't representative of what's to come? In this section we want to prompt you to come up with predictive elements to improve our models. Feel free to use some of the ideas below or go in an entirely new direction!

In this section, we encourage you to think creatively about enhancing predictive models for climate-related disasters. Consider how to incorporate novel data sources, feature engineering techniques, and model architectures to improve predictions. Below are some suggested approaches, but feel free to explore entirely new directions!

### Case Study Focus: Coastal Eastern African Nations

Using the disaster impact data for coastal Eastern African nations, can you develop a model to predict how these impacts might escalate for Madagascar in the coming years? Consider not only the historical data but also factors such as changes in sea surface temperatures, shifting storm tracks, population growth along vulnerable coastlines, and evolving infrastructure resilience. Further, can you integrate this predictive model into an optimization framework to better allocate resources for disaster preparedness and response?

Potential Approaches to Explore:
- Comparing Time Series Models: Traditional statistical time series models like ARIMAX (AutoRegressive Integrated Moving Average with Explanatory Variables) are commonly used to predict future values based on past data. How do these models compare with more advanced Recurrent Neural Network (RNN)-based approaches like Long Short-Term Memory (LSTM) networks or Gated Recurrent Units (GRUs) in capturing long-term dependencies, especially under non-stationary conditions induced by climate change?

- Incorporating Geospatial Data: Geospatial features, such as latitude, longitude, elevation, and proximity to bodies of water, can play a crucial role in predicting the impact of tropical storms. Can we encode geospatial information using techniques like convolutional neural networks (CNNs) for spatial feature extraction, or leverage more specialized models such as Geographical Weighted Regression (GWR) or Graph Neural Networks (GNNs) to account for spatial dependencies?

- Incorporating Climate Change Projections: Beyond just historical data, consider how future climate projections can be integrated into the model. Can we use downscaled climate model outputs or ensemble approaches to account for different climate scenarios? How would these scenarios affect the frequency and intensity of tropical storms affecting coastal Eastern African nations?

- Feature Engineering with Climate Indicators: Introduce climate change indicators as predictive features. For example, how do trends in sea surface temperatures (SSTs), El Niño-Southern Oscillation (ENSO) phases, or the Atlantic Multi-decadal Oscillation (AMO) correlate with storm intensification? Would incorporating these indicators as additional time series variables enhance predictive accuracy?
	
- Hybrid and Ensemble Models: Can we leverage hybrid models that combine both statistical and deep learning approaches, or use ensemble methods that aggregate predictions from multiple models? For example, combining ARIMAX for short-term predictions with LSTM for capturing long-term trends may provide a more comprehensive forecasting tool.

- Optimization Integration: Once a reliable predictive model is established, how can it be integrated into an optimization framework for resource allocation? For example, can we build an optimization model that minimizes both the cost of disaster preparedness and the potential loss from future storm impacts?

- Model Explainability and Decision-Making: How can we ensure that the model is interpretable for decision-makers? Consider the use of techniques like SHAP (SHapley Additive exPlanations) values or LIME (Local Interpretable Model-agnostic Explanations) to explain which factors contribute most to the model’s predictions, helping policymakers make informed decisions.

By combining predictive analytics with optimization, we can not only forecast future disaster impacts but also develop actionable strategies for minimizing those impacts. The goal is to make our models both more accurate and more useful in real-world applications, driving better outcomes for communities at risk.
